In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
import time
import pickle
import pandas as pd
from datetime import datetime
import os
import scrapy
from bs4 import BeautifulSoup
import json

In [2]:
options = Options()
options.add_argument("--start-maximized")  # Open Edge in maximized mode
options.add_experimental_option("excludeSwitches", ["enable-automation"])

edge_path = r"D:\webtool\edgedriver_win64\msedgedriver.exe"
service = Service(edge_path)

driver = webdriver.Edge(service=service, options=options)

In [3]:
default_sub_time = datetime(1980, 1, 1, 0, 0, 0)
print(default_sub_time)

1980-01-01 00:00:00


In [4]:
with open("last_time.txt", "w") as f:
    f.write("1980-01-01 00:00:00")

In [5]:
if not os.path.exists("last_time.txt"):
    with open("last_time.txt", "w") as f:
        f.write("1980-01-01 00:00:00")
else:
    with open("last_time.txt", "r") as f:
        last_time = datetime.strptime(f.read().strip(), "%Y-%m-%d %H:%M:%S")

In [6]:
last_time

datetime.datetime(1980, 1, 1, 0, 0)

In [7]:
def transform_str_date(sub_time_str : str) -> datetime:
    return datetime.strptime(sub_time_str.strip(), "%H:%M | %d/%m/%Y")


def check_time_new(sub_time_date : datetime) -> bool:
    return max(sub_time_date, last_time)

In [8]:
time_test = transform_str_date("23:36 | 18/04/2025")

In [9]:
check_time_new(time_test)

datetime.datetime(2025, 4, 18, 23, 36)

In [10]:
DOMAIN = "https://thuvienphapluat.vn"

In [11]:
def remove_link_tag(link) -> bool:
    if link.startswith("/phap-luat/tag/"):
        return False
    return True


def cleanText(text):
    text = text.strip()
    text = text.replace("\n", "")
    text = text.replace("\r", "")
    return text

In [12]:
first_page = 1
last_page = 1
start_urls = ["https://thuvienphapluat.vn/phap-luat/bat-dong-san?page={}".format(
    i) for i in range(first_page, last_page + 1)]
print(start_urls)

['https://thuvienphapluat.vn/phap-luat/bat-dong-san?page=1']


In [24]:
index_question = 1
data = pd.DataFrame(
    columns=["index_question", "question", "url", "tags", "created_date"])

In [ ]:
def crawl_URL_Question(data: pd.DataFrame, url: str) -> pd.DataFrame:
    global index_question
    """ Crawl câu hỏi từ trang web và trả về DataFrame """
    driver.get(url)
    time.sleep(1)
    temp_list = []  # Thay vì dùng `temp_df.append`, ta dùng list để tăng hiệu suất
    try:
        articles = driver.find_elements(
            By.TAG_NAME, "article")
        for article in articles:
            try:
                a_tag = article.find_element(By.TAG_NAME, 'a')
                if a_tag:
                    url_answer = a_tag.get_attribute('href')
                    question = a_tag.get_attribute("title").strip()
                    tags = a_tag.find_elements(By.TAG_NAME, "span")
                    tag_list = []
                    for tag in tags:
                        tag_list.append(tag.text)
                    created_date_elems = article.find_elements(By.CLASS_NAME, "sub-time")
                    created_date = created_date_elems[0].text if created_date_elems else "N/A"
                else:
                    continue
                temp_list.append({
                    "index_question": index_question,
                    "question": question,
                    "url": url_answer,
                    "tags": tag_list,
                    "created_date": created_date,
                })
                print(f"Index: {index_question}")
                print(f"Câu hỏi: {question}")
                print(f"URL: {url_answer}")
                print(f"Tags: {tag_list}")
                print(f"Created Date: {created_date}")
                print("------------------------------------")
                index_question += 1
            except NoSuchElementException:
                continue
            except Exception as e:
                print(f"Lỗi khi lấy dữ liệu: {e}")

    except Exception as e:
        print(f"Lỗi khi truy cập trang {url}: {e}")

    # Dùng pd.concat thay vì append để tối ưu hiệu suất
    temp_df = pd.DataFrame(temp_list, columns=[
                           "index_question", "question", "url", "tags", "created_date"])
    data = pd.concat([data, temp_df], ignore_index=True)
    return data

In [26]:
for start_url in start_urls:
    print(start_url)
    data = crawl_URL_Question(data, start_url)

https://thuvienphapluat.vn/phap-luat/bat-dong-san?page=1
Index: 1
Câu hỏi: Đã có Nghị quyết 99 NQ CP 2025 điều chỉnh Quy hoạch sử dụng đất quốc gia thời kỳ 2021 2030?
URL: https://thuvienphapluat.vnhttps://thuvienphapluat.vn/phap-luat/ho-tro-phap-luat/da-co-nghi-quyet-99-nq-cp-2025-dieu-chinh-quy-hoach-su-dung-dat-quoc-gia-thoi-ky-2021-2030-543772-211741.html
Tags: ['Đã có Nghị quyết 99 NQ CP 2025 điều chỉnh Quy hoạch sử dụng đất quốc gia thời kỳ 2021 2030?']
Created Date: 15:05 | 19/04/2025
------------------------------------
Index: 2
Câu hỏi: Nhà ở hình thành trong tương lai có phải dự án bất động sản đưa vào kinh doanh? Điều kiện của nhà ở hình thành trong tương lai được đưa vào kinh doanh là gì?
URL: https://thuvienphapluat.vnhttps://thuvienphapluat.vn/phap-luat/nha-o-hinh-thanh-trong-tuong-lai-co-phai-du-an-bat-dong-san-dua-vao-kinh-doanh-dieu-kien-cua-nha-o--59924-211362.html
Tags: ['Nhà ở hình thành trong tương lai có phải dự án bất động sản đưa vào kinh doanh? Điều kiện của nh

In [27]:
data

,index_question,question,url,tags,created_date
0,1,Đã có Nghị quyết 99 NQ CP 2025 điều chỉnh Quy ...,https://thuvienphapluat.vnhttps://thuvienphapl...,[Đã có Nghị quyết 99 NQ CP 2025 điều chỉnh Quy...,15:05 | 19/04/2025
1,2,Nhà ở hình thành trong tương lai có phải dự án...,https://thuvienphapluat.vnhttps://thuvienphapl...,[Nhà ở hình thành trong tương lai có phải dự á...,09:04 | 19/04/2025
2,3,Có được phép chứng thực chữ ký trên giấy ủy qu...,https://thuvienphapluat.vnhttps://thuvienphapl...,[Có được phép chứng thực chữ ký trên giấy ủy q...,23:36 | 18/04/2025
3,4,Block của tòa nhà chung cư được hiểu thế nào? ...,https://thuvienphapluat.vnhttps://thuvienphapl...,[Block của tòa nhà chung cư được hiểu thế nào?...,19:39 | 18/04/2025
4,5,Khi kinh doanh bất động sản có được không công...,https://thuvienphapluat.vnhttps://thuvienphapl...,[Khi kinh doanh bất động sản có được không côn...,06:41 | 18/04/2025
5,6,"Nguyên tắc lập quy hoạch, kế hoạch sử dụng đất...",https://thuvienphapluat.vnhttps://thuvienphapl...,"[Nguyên tắc lập quy hoạch, kế hoạch sử dụng đấ...",03:10 | 18/04/2025
6,7,"Hướng dẫn chỉnh lý, cấp sổ đỏ mới sau sáp nhập...",https://thuvienphapluat.vnhttps://thuvienphapl...,"[Hướng dẫn chỉnh lý, cấp sổ đỏ mới sau sáp nhậ...",20:20 | 17/04/2025
7,8,Công bố dự án đầu tư xây dựng nhà ở xã hội l...,https://thuvienphapluat.vnhttps://thuvienphapl...,[Công bố dự án đầu tư xây dựng nhà ở xã hội ...,19:39 | 17/04/2025
8,9,Hợp đồng thuê nhà ở có thuộc các loại hợp đồng...,https://thuvienphapluat.vnhttps://thuvienphapl...,[Hợp đồng thuê nhà ở có thuộc các loại hợp đồn...,16:16 | 17/04/2025
9,10,Thể lệ Cuộc thi tìm hiểu lịch sử chào mừng kỷ ...,https://thuvienphapluat.vnhttps://thuvienphapl...,[Thể lệ Cuộc thi tìm hiểu lịch sử chào mừng kỷ...,15:19 | 17/04/2025


In [28]:
data.iloc[18]

index_question                                                   19
question          Có phải đổi sang sổ đỏ mẫu mới khi sáp nhập tỉ...
url               https://thuvienphapluat.vnhttps://thuvienphapl...
tags              [Có phải đổi sang sổ đỏ mẫu mới khi sáp nhập t...
created_date                                     14:34 | 15/04/2025
Name: 18, dtype: object